# МОиВС "Генеративные модели", 5-й модуль

# Homework 1

В этой домашней работе вам предстоит добавить к BERT'у декодерную часть и решить задачу генерации суммаризаций для текстов новостей на русском языке.

Дополнительно к этому на отличную оценку потребуется реализовать подсчет метрик качества и менее жадную стратегию выбора следующего токена для генерации.

*Мы сразу вас предостерегаем попасть в петлю бесконечного дообучения модели. Эта домашка не на пробитие скора. Мы будем проверять, что вы, в целом, сделали все верно и смогли получить какую-то более-менее адекватную (такую, которая заметно лучше той, что была до начала обучения) генерацию. Таким образом, если вы видите, что модель учится, не надо дообучать её сутками. Нескольких часов точно должно хватить.*



---


---
По любым вопросам касательно этой домашней работы обращайтесь ко своим ассистентам




In [ ]:
%%bash
python3 -m pip install transformers datasets evaluate

In [7]:
import math
import warnings
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import evaluate
import matplotlib.pyplot as plt


from tqdm import tqdm
from IPython.display import clear_output
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel, AutoTokenizer
from datasets import load_dataset



warnings.filterwarnings('ignore', category=FutureWarning)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Подготовка данных (0.5 балла)

Мы воспользуемся датасетом с 🤗 Ильи Гусева "gazeta". Он представляет собой пары (полный текст новости -- его саммари). Пары были взяты с одноименного сайта в домене .ru

Более подробно про датасет можно прочитать [здесь](https://huggingface.co/datasets/IlyaGusev/gazeta)



In [2]:
# Загрузим данные с попощью библиотеки библиотеки datasets

dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0", split='train[:5%]')

Вы должны помнить, что тексты перед подачей в модель необходимо **токенизировать**.

Добавьте паддинг до `max_length=512` для обучающих данных, а также до `max_length=128` для меток.

Используйте обрезку текстов, длина которых в токенах превышает `max_length`

In [3]:
# Подготовим данные для модели Bert

model_name = 'deepvk/bert-base-uncased' # Указание модели BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):

    # токенайзер для текста
    model_inputs = tokenizer(
        examples['text'], 
        max_length= 512,
        truncation= True,
        padding= 'max_length'
        )
    # токенайзер для меток
    labels = tokenizer(
        examples['summary'],
        max_length= 128,
        truncation= True,
        padding= 'max_length' 
        )
    
    model_inputs['labels'] = labels["input_ids"]
    return model_inputs

In [4]:
tokenized_dataset = dataset.map(preprocess, batched=False)
tokenized_dataset.set_format('torch')

Размер батча советуем подбирать таким образом, чтоб утилизировать максимум доступной VRAM

In [5]:
split_data = tokenized_dataset.train_test_split(test_size = 0.2)

train_dataloader = DataLoader(split_data['train'], batch_size= 8, shuffle= True)
eval_dataloader = DataLoader(split_data['test'], batch_size= 8)

## Реализация Decoder-cети (3 балла)

В данном разделе вам необходимо **реализовать собственный декодер для генерации текста**.

Можете вдохновляться кодом с семинара 1 по GPT. В инициализации весов стоит (но необязательно) проявить смекалку

In [6]:
# Класс модели для суммаризации на основе BERT с кастомным декодером

class BertSummarizer(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_size=768, num_decoder_layers=3, num_heads=8):
        super(BertSummarizer, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hidden_size = hidden_size

        # Эмбеддинги для токенов на входе в декодер
        self.embedding = nn.Embedding(
            num_embeddings = self.bert.config.vocab_size,
            embedding_dim = self.hidden_size
            )

        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model = self.hidden_size,
            nhead = num_heads
            )
        
        self.decoder = nn.TransformerDecoder(
            decoder_layer = self.decoder_layer,
            num_layers = num_decoder_layers
            )
        
        self.fc_out = nn.Linear(self.hidden_size, self.bert.config.vocab_size)
        self.softmax = nn.Softmax()
        

    # Функция для создания маски для предотвращения заглядывания вперед в декодере
    def generate_square_subsequent_mask(self, T):
    
        result = torch.zeros(T, T)
        mask = torch.triu(torch.ones(T, T), diagonal=1)
        result[mask.bool()] = float('-inf')

        return result


    def forward(self, input_ids, attention_mask, decoder_input_ids):
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state  # Выходы BERT для использования в декодере

        # Эмбеддинги для входных токенов декодера
        embedded = self.embedding(decoder_input_ids)

        tgt_mask = self.generate_square_subsequent_mask(embedded.size(1))#.to(input_ids.device)
        decoded = self.decoder(tgt=embedded.transpose(0, 1), memory = memory.transpose(0, 1), tgt_mask= tgt_mask)
        output = self.fc_out(decoded.transpose(0, 1))

        return self.softmax(output)
    

    def generate(self, input_ids, attention_mask, tokenizer, max_len= 50):
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state
        batch_size = input_ids.size(0)

        # Начинаем с токена [CLS] или [BOS] (начало последовательности)
        decoder_input_ids = torch.full((batch_size, 1), tokenizer.cls_token_id, dtype=torch.long).to(input_ids.device)
        memory = memory.transpose(0, 1)
        # generated_tokens = []

        for _ in range(max_len):
            embedded = self.embedding(decoder_input_ids).transpose(0, 1)

            # Генерация маски для предотвращения заглядывания вперед
            decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(0)).to(input_ids.device)
            decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)

            output = self.fc_out(decoder_output.transpose(0, 1))

            # Получаем индекс токена с наибольшей вероятностью.
            # Помните, если EOS предсказан, прекращаем генерацию

            gen_token = torch.argmax(output[:, -1, :], dim=-1, keepdim=True)
            decoder_input_ids = torch.cat([decoder_input_ids, gen_token], dim=-1)
            
            if tokenizer.sep_token_id in gen_token:
                break

        generated_sequence = tokenizer.decode(
            decoder_input_ids.squeeze().tolist(),
            skip_special_tokens=True
            )

        return generated_sequence
    


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=50):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    


# Инициализируем нашу модель и посморим на ее архитектруру
model = BertSummarizer(bert_model_name=model_name)
model = model.to(device)

In [77]:
# Инициализируем нашу модель и посморим на ее архитектруру

# model = BertSummarizer(bert_model_name=model_name)
# model = model.to(device)
# model

In [100]:
# Посмотрим на генерацию без обучения

eval_data_sample = next(iter(eval_dataloader))
model.generate(eval_data_sample['input_ids'][:1].to(device), eval_data_sample['attention_mask'][:1].to(device), tokenizer)

'алексеи интересная эксп юноши всп встали западу организ ##ительных ##оо феис продавец егер правило недавнего безумныи нечего идио ##зовом ##авшеи вспомнил ##хова сверст сверст фаши натяжные клад звездо приемлем подвиги проиграл любая математике ввс танцев властеи трансформи ##рец натяжные клад звездо приемлем подвиги драгоцен ##товид ##щеи ##тие toyota сохранить о_о'

## Обучение модели (1 балл)

<small> 0.25 балла за простейший рабочий цикл; </small>

<small> +0.5 балла за графики для лосса и метрик на трейне и валидации.</small>

В данном разделе вам необходимо **реализовать цикл для обучения модели**


In [102]:
# Пример обучения на одной итерации

def train_step(model, input_ids, attention_mask, decoder_input_ids, optimizer, criterion):
    
    model.train()
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask, decoder_input_ids)
    loss = criterion(outputs.view(-1, outputs.size(-1)), decoder_input_ids.view(-1))
    loss.backward()
    optimizer.step()

    return loss.item()


def plot_losses(train_losses, val_losses):
    '''
    Plot losses and metrics while training
    - train_losses: sequence of train losses
    - val_losses: sequence of validation losses
    '''
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(val_losses) + 1), val_losses, label='val')

    if max(train_losses) / min(train_losses) > 10:
        axs[0].set_yscale('log')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    axs[0].set_ylabel('loss')
    axs[1].set_ylabel('MSE')
    plt.show()

In [103]:
optimizer = torch.optim.AdamW(model.parameters(), lr= 1e-3)
num_epochs = 1
criterion = nn.CrossEntropyLoss(ignore_index=0)


train_losses = []
eval_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for batch in tqdm(train_dataloader, desc= f'Epoch: {epoch + 1}'):
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        shifted_labels = torch.cat(
            [labels[:, 1:], torch.full((labels.size(0), 1), tokenizer.pad_token_id).to(device)], 
            dim = 1
            )
        
        optimizer.zero_grad()

        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_input_ids = shifted_labels
            )
        
        loss = criterion(
            outputs.view(-1, outputs.size(-1)),
            labels.view(-1)
            )

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_dataloader))
    model.eval()
    running_loss = 0
    for batch in tqdm(eval_dataloader, desc= f'Epoch: {epoch + 1}'):
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        shifted_labels = torch.cat(
            [labels[:, 1:], torch.full((labels.size(0), 1), tokenizer.pad_token_id).to(device)], 
            dim = 1
            )
        with torch.no_grad():

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                decoder_input_ids = shifted_labels
                )
            
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                labels.view(-1)
                )

            running_loss += loss.item()
    
    eval_losses.append(running_loss / len(eval_dataloader))
    plot_losses(train_losses, eval_losses)
    

Epoch: 1:   0%|          | 0/305 [00:00<?, ?it/s]/Users/ivanyuminov/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Epoch: 1:   3%|▎         | 8/305 [03:18<2:02:59, 24.85s/it]


KeyboardInterrupt: 

## Метрики качества (1 балл)

<small>По 0.33 балла за реализацию каждой из предлагаемых метрик</small>

**Реализуйте функицию для подсчета метрик качества суммаризации.**

Докуметация по некотрым метрикам:
 1. [HuggingFace Rouge](https://huggingface.co/spaces/evaluate-metric/rouge)
 2. [HuggingFace Bleu](https://huggingface.co/spaces/evaluate-metric/bleu)
 3. [HuggingFace BERT Score](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [34]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def compute_metrics():
    #<YOUR CODE HERE>
    pass

def evaluation():
    #<YOUR CODE HERE>
    pass

## Обучение модели (0.5 балла)
**Обучите модель, сохраните лучшую версию** (метод `.save_pretrained()` объекта класса AutoModel... или `torch.save()`) **и добавьте пример генерации**. Учтите, что если изменялся токенизатор (а лучше просто по умолчанию), его тоже нужно сохранить. Если планируете продолжить обучение

Для сравнения оценки качества генерации по значениям реализованных метрик можете запустить ruT5-small без дообучения. Мы намеренно даем бейзлайн именно в таком виде.

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("YOUR MODEL")
summary = #<YOUR CODE HERE>

## Реализация менее жадных стратегий выбора следующего токена (4 балла)
Всегда ли выбор наиболее вероятного токена на каждом шаге – это лучшая стратегия для генерации текста?

<details>
    <summary>Спойлер</summary>
    <p>Нет</p>
</details>

**Сравнение стратегий для генерации текста:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br><br/> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br><br/> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words <br/> that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br><br/> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps <br/> the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br><br/> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br><br/> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities <br/> by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br><br/> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br><br/> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br><br/> **Cons:** Computationally more complex and requires a good loss function. |

Ссылки на докуметацию:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

**1. Дополните метод `generate` в модели, чтобы получать топ-k самых вероятных токена и их "вероятности"** (1 балл).   

**2. Реализуйте стратегию Nucleus Sampling в методе `generate`** (1 балл)

**3. Реализуйте стратегию Beam Search** (2 балла)

Получилось ли улучшить генерацию?

## Послевкусие (0 баллов)

Если эта домашняя работа показалась вам недостаточно большой, предлагаем провести следующий эксперимент:

- от имеющейся модели "откусить" только декодерную часть (откусить также можно от ruT5-small);
- немного дообучить (что называется, по вкусу);
- посмотреть качество генерации по метрикам и "глазами";
- сравнить полученное с Encoder-Decoder архитектурой;
- ответить на вопрос "Дает ли применение Encoder-Decoder архитектуры значительный буст в качестве генерации, или это некоторый overkill?" (базово, ответ лежит на поверхности 😸)

Ещё более опционально можно:
- почитать про возможности генерации Encoder-only архитектурными решениями (BERT, e.g.)
- сравнить с генерацией только Decoder'ом и both Encoder-Decoder'ом;
- в т.ч. подобрать число обучаемых параметров таким образом, чтоб оно было примерно одинаковым для каждого инстанса моделей (их, инстансов, будет 3 -- только энкодер, только декодер и энкодер-декодер).

*Вообще ориентироваться следует на следующее утверждение: "Только энкодерные архитектуры (BERT, e.g.) хороши для понимания текста (получения эмеддингов), лишь декодерные (GPT, например) -- для генерации, энкодер-декодерные (скажем, T5) -- для обеих задач"*